In [1]:
import pandas as pd
from models import *
from tqdm.autonotebook import tqdm
tqdm.pandas()
from torch import nn
import json
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from argparse import Namespace
from transformers import *
import torch
import matplotlib.pyplot as plt
import torch.utils.data
import torch.nn.functional as F
import argparse
from transformers.modeling_utils import * 
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from vncorenlp import VnCoreNLP
from utils import *

In [2]:
args = Namespace(
    dict_path = "./phobert_base/dict.txt",
    config_path = "./phobert_base/config.json",
    rdrsegmenter_path = '/home/tuna/FDM/MarketSentiment/PhoBert-Sentiment-Classification/VnCoreNLP-master/VnCoreNLP-1.1.1.jar',
    pretrained_path = './phobert_base/model.bin',
    max_sequence_length = 256,
    batch_size = 24,
    accumulation_steps = 5,
    epochs = 15, # 5
    fold = 0,
    seed = 69,
    lr = 3e-6, # 3e-5
    ckpt_path = './models',
    bpe_codes = "./phobert_base/bpe.codes"
)

In [3]:
seed_everything(69)
bpe = fastBPE(args)
rdrsegmenter = VnCoreNLP(args.rdrsegmenter_path, annotators="wseg", max_heap_size='-Xmx500m') 

./phobert_base/bpe.codes


In [4]:
# Load model
config = RobertaConfig.from_pretrained(
    args.config_path,
    output_hidden_states=True,
    num_labels=1
)
print(config)
model_bert = RobertaForAIViVN.from_pretrained(args.pretrained_path, config=config)
model_bert.cuda()
tsfm = model_bert.roberta

RobertaConfig {
  "_num_labels": 1,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 258,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "prefix": null,
  "pr

In [5]:
# Load the dictionary  
vocab = Dictionary()
vocab.add_from_file(args.dict_path)
print(vocab.indices)

{'<s>': 0, '<pad>': 1, '</s>': 2, '<unk>': 3, ',': 4, '.': 5, 'và': 6, 'của': 7, 'là': 8, 'các': 9, 'có': 10, 'được': 11, 'trong': 12, 'cho': 13, 'đã': 14, 'với': 15, 'một': 16, 'không': 17, 'người': 18, ')': 19, '(': 20, 'những': 21, '"': 22, 'này': 23, 'để': 24, 'ở': 25, 'khi': 26, ':': 27, 'về': 28, 'năm': 29, 'đến': 30, '-': 31, 'cũng': 32, 'vào': 33, 'trên': 34, 'tại': 35, 'nhiều': 36, 'đó': 37, 'sẽ': 38, 'từ': 39, 'ra': 40, 'phải': 41, 'như': 42, 'ngày': 43, 'lại': 44, 'bị': 45, 'ông': 46, 'làm': 47, 'hơn': 48, 'việc': 49, 'còn': 50, 'nhưng': 51, 'đang': 52, 'sau': 53, 'thì': 54, 'biết': 55, 'Việt_Nam': 56, 'đi': 57, 'nước': 58, 'rất': 59, 'mới': 60, 'sự': 61, 'có_thể': 62, 'theo': 63, 'mà': 64, ';': 65, 'chỉ': 66, 'nhất': 67, 'mình': 68, 'nhà': 69, 'tôi': 70, 'trước': 71, 'lên': 72, 'con': 73, 'vẫn': 74, 'tới': 75, '2': 76, 'nên': 77, 'tháng': 78, 'Theo': 79, 'đồng': 80, 'cùng': 81, 'hai': 82, 'anh': 83, 'cao': 84, 'khác': 85, 'họ': 86, 'rằng': 87, 'bạn': 88, 'qua': 89, 'vì': 90

In [6]:
train_df = pd.read_csv('Sentiment_result1.csv')
train_df = train_df[['content', 'sentiment_final']]
train_df = train_df.rename(columns={'content':'text'})
train_df = train_df.loc[train_df['sentiment_final'].isin(['P', 'N'])]
train_df = train_df.drop_duplicates(keep='first').reset_index(drop=True)
print(train_df)
print(train_df.sentiment_final.value_counts())
train_df.to_csv('data_clean.csv', sep='#', index=False)
train_df.text = train_df.text.progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(x)]))
X_train = convert_lines(train_df, vocab, bpe,args.max_sequence_length)
print(X_train)
print(X_train.shape)
#
label_encoder = LabelEncoder() 
train_df['sentiment_final']= label_encoder.fit_transform(train_df['sentiment_final']) 
y = train_df.sentiment_final.values
print(y)

  0%|          | 0/430 [00:00<?, ?it/s]

                                                  text sentiment_final
0    Xin thông báo khi nào hết dịch Corona thì lúc ...               P
1    5 mã trắng cửa bán rồi thì FLC tí nữa thôi là ...               P
2    Hihi theo a Quyết đến hết mùa dịch chắc cũng p...               P
3    Mai tây (ETFs) nó xả hàng ROS, mà a QC còn chơ...               N
4    Art, Hai, KLF lái con chốt, a e vô đỡ thấy thư...               N
..                                                 ...             ...
425  Tin vĩ mô:.- Bộ TC gia tăng gói hỗ trợ một số ...               P
426  T2 này khả năng thoát dớp nhưng về lâu dài phe...               N
427                             Giảm thêm 700 đáy chắc               N
428  Cuối tuần ngập tràn tin xấu.Cá Mập, đội lái đã...               N
429                                Vni lên là lên !!!!               P

[430 rows x 2 columns]
P    273
N    157
Name: sentiment_final, dtype: int64


100%|██████████| 430/430 [00:00<00:00, 2404.76it/s]

[[6.3117e+04 1.3020e+03 8.8400e+02 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [6.3117e+04 1.3020e+03 8.8400e+02 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [6.3117e+04 1.3020e+03 8.8400e+02 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 ...
 [6.3117e+04 1.3020e+03 8.8400e+02 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [6.3117e+04 1.3020e+03 8.8400e+02 ... 6.1320e+03 7.1510e+03 2.0000e+00]
 [6.3117e+04 1.3020e+03 8.8400e+02 ... 1.0000e+00 1.0000e+00 1.0000e+00]]
(430, 128)
[1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1
 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0
 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0
 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1
 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1

In [7]:
# Creating optimizer and lr schedulers
param_optimizer = list(model_bert.named_parameters())
print(len(param_optimizer))
print('param_optimizer', param_optimizer)
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
print('optimizer_grouped_parameters', optimizer_grouped_parameters)
num_train_optimization_steps = int(args.epochs*len(train_df)/args.batch_size/args.accumulation_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=args.lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
scheduler0 = get_constant_schedule(optimizer)  # PyTorch scheduler

201
param_optimizer [('roberta.embeddings.word_embeddings.weight', Parameter containing:
tensor([[ 0.0472, -0.0314,  0.0343,  ...,  0.0144, -0.0032, -0.0680],
        [ 0.0186, -0.0044,  0.0190,  ...,  0.0169, -0.0055, -0.0302],
        [-0.0074, -0.0078, -0.0142,  ...,  0.0177, -0.0011,  0.0023],
        ...,
        [ 0.0374, -0.0152,  0.0123,  ...,  0.0210, -0.0214, -0.0185],
        [ 0.0266,  0.0122, -0.0232,  ...,  0.0253,  0.0002, -0.0303],
        [ 0.0106, -0.0063, -0.0064,  ...,  0.0092, -0.0129, -0.0153]],
       device='cuda:0', requires_grad=True)), ('roberta.embeddings.position_embeddings.weight', Parameter containing:
tensor([[ 0.0150, -0.0027, -0.0069,  ..., -0.0021,  0.0077,  0.0108],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0401, -0.0189, -0.0934,  ..., -0.0437,  0.0582,  0.0378],
        ...,
        [ 0.0010, -0.0455, -0.0163,  ..., -0.0149,  0.0101,  0.0913],
        [-0.0101, -0.0156,  0.0074,  ..., -0.0072, -0.0033,  0.132

optimizer_grouped_parameters [{'params': [Parameter containing:
tensor([[ 0.0472, -0.0314,  0.0343,  ...,  0.0144, -0.0032, -0.0680],
        [ 0.0186, -0.0044,  0.0190,  ...,  0.0169, -0.0055, -0.0302],
        [-0.0074, -0.0078, -0.0142,  ...,  0.0177, -0.0011,  0.0023],
        ...,
        [ 0.0374, -0.0152,  0.0123,  ...,  0.0210, -0.0214, -0.0185],
        [ 0.0266,  0.0122, -0.0232,  ...,  0.0253,  0.0002, -0.0303],
        [ 0.0106, -0.0063, -0.0064,  ...,  0.0092, -0.0129, -0.0153]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([[ 0.0150, -0.0027, -0.0069,  ..., -0.0021,  0.0077,  0.0108],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0401, -0.0189, -0.0934,  ..., -0.0437,  0.0582,  0.0378],
        ...,
        [ 0.0010, -0.0455, -0.0163,  ..., -0.0149,  0.0101,  0.0913],
        [-0.0101, -0.0156,  0.0074,  ..., -0.0072, -0.0033,  0.1323],
        [-0.0528,  0.0500, -0.1011,  ...,  0.0354, -0.0190,  0.1747]],
 

In [8]:
model_bert

RobertaForAIViVN(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, 

In [9]:
if not os.path.exists(args.ckpt_path):
    os.mkdir(args.ckpt_path)

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X_train, y))
for fold, (train_idx, val_idx) in enumerate(splits):
    print("Training for fold {}".format(fold))
    best_score = 0
    if fold != args.fold:
        continue
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))
    tq = tqdm(range(args.epochs + 1))
    for child in tsfm.children():
        for param in child.parameters():
            if not param.requires_grad:
                print("whoopsies")
            param.requires_grad = False
    frozen = True
    for epoch in tq:
        if epoch > 0 and frozen:
            for child in tsfm.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
#             del scheduler0
            torch.cuda.empty_cache()
        train_preds = []
        val_preds = []
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False)
        avg_loss = 0.
        avg_accuracy = 0.
        optimizer.zero_grad()
        pbar = tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
        for i,(x_batch, y_batch) in pbar:
            model_bert.train()
            y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
            loss =  F.binary_cross_entropy_with_logits(y_pred.view(-1).cuda(),y_batch.float().cuda())
            loss = loss.mean()
            loss.backward()
            if i % args.accumulation_steps == 0 or i == len(pbar) - 1:
                optimizer.step()
                optimizer.zero_grad()
                if not frozen:
                    scheduler.step()
                else:
                    scheduler0.step()
            lossf = loss.item()
            pbar.set_postfix(loss = lossf)
            avg_loss += loss.item() / len(train_loader)
            y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
            y_pred = y_pred.squeeze().detach().cpu().numpy()
            train_preds = np.concatenate([train_preds, np.atleast_1d(y_pred)])
        train_preds = sigmoid(train_preds)
#         print(train_preds)
        model_bert.eval()
        pbar = tqdm(enumerate(valid_loader),total=len(valid_loader),leave=False)
        for i,(x_batch, y_batch) in pbar:
            y_pred = model_bert(x_batch.cuda(), attention_mask=(x_batch>0).cuda())
            y_pred = y_pred.squeeze().detach().cpu().numpy()
            val_preds = np.concatenate([val_preds, np.atleast_1d(y_pred)])
        val_preds = sigmoid(val_preds)
        print(val_preds)
        best_th = 0
        socre_train = f1_score(y[train_idx], train_preds > 0.5)
        score = f1_score(y[val_idx], val_preds > 0.5)
        print(f"\nAUC = {roc_auc_score(y[train_idx], train_preds):.4f}, F1 score @0.5 = {socre_train:.4f}")
        print(f"\nAUC = {roc_auc_score(y[val_idx], val_preds):.4f}, F1 score @0.5 = {score:.4f}")
        if score >= best_score:
            print("Saving model")
            torch.save(model_bert.state_dict(),os.path.join(args.ckpt_path, f"model_{fold}.bin"))
            best_score = score

  0%|          | 0/15 [00:00<?, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)

  0%|          | 0/15 [00:00<?, ?it/s, loss=0.708]

Training for fold 0



 93%|█████████▎| 14/15 [00:01<00:00,  7.81it/s, loss=0.666]
                                                           
100%|██████████| 4/4 [00:00<00:00, 17.32it/s]
                                             

[0.55658928 0.5791961  0.57537051 0.51060704 0.56624914 0.5725307
 0.57666678 0.55765698 0.57896699 0.55583484 0.58350795 0.55969614
 0.54995509 0.57705326 0.57311966 0.553829   0.57311311 0.57459753
 0.58331793 0.57396458 0.41832551 0.56030665 0.56102739 0.57284095
 0.5539243  0.55325598 0.54274442 0.57275263 0.56920565 0.56306136
 0.57756993 0.55555955 0.54500727 0.56517888 0.54339151 0.57555735
 0.56380773 0.56328184 0.56146927 0.55635234 0.57709602 0.56673967
 0.5419765  0.55646711 0.55930055 0.56645025 0.56147557 0.57400188
 0.53069344 0.55368736 0.56343803 0.49640356 0.56825565 0.56318058
 0.5627879  0.53942873 0.57457324 0.57520445 0.5540418  0.58038353
 0.58074751 0.5572515  0.58408241 0.56468493 0.56972357 0.48819147
 0.53998823 0.55581108 0.56946639 0.55233273 0.5595697  0.57517564
 0.57151102 0.55975363 0.5665622  0.56411162 0.57601136 0.56259063
 0.56985438 0.44838721 0.58009639 0.56990031 0.58375678 0.55810913
 0.56565436 0.56650465]

AUC = 0.4477, F1 score @0.5 = 0.7551



100%|██████████| 15/15 [00:03<00:00,  4.78it/s, loss=0.666]
                                                           
100%|██████████| 4/4 [00:00<00:00, 17.04it/s]
                                             

...

[0.57746215 0.59272534 0.58492391 0.51482127 0.58328495 0.58551967
 0.5890927  0.57147924 0.58572509 0.5744698  0.5889571  0.56389638
 0.5647856  0.58872859 0.58152707 0.57738203 0.58075246 0.59145538
 0.59158181 0.58426654 0.42372984 0.57761419 0.57977698 0.58237644
 0.56959717 0.56519113 0.5648894  0.58612969 0.58224211 0.57104087
 0.58769445 0.57093771 0.56588269 0.57290999 0.56456768 0.59067521
 0.57918832 0.57185728 0.57639098 0.56969762 0.58594426 0.57922843
 0.55520519 0.56792247 0.57785278 0.57810528 0.58140229 0.5827724
 0.53531878 0.56328526 0.57356301 0.50133755 0.57843959 0.57595955
 0.57430381 0.56410178 0.5840682  0.58369811 0.56862873 0.58708732
 0.5883823  0.56619182 0.59973701 0.57452156 0.57119676 0.49304512
 0.56874245 0.56693885 0.58470354 0.57078232 0.57199186 0.59165766
 0.58697272 0.566158   0.57957487 0.57092255 0.57915742 0.58244372
 0.5794202  0.45516568 0.59335481 0.58487429 0.59258695 0.56331887
 0.57872492 0.57285227]

AUC = 0.4679, F1 score @0.5 = 0.7523




100%|██████████| 15/15 [00:03<00:00,  4.59it/s, loss=0.663]
                                                           
  0%|          | 0/15 [00:00<?, ?it/s]

[0.57974522 0.59419478 0.58605591 0.5153744  0.58507023 0.58677645
 0.59031206 0.57309658 0.58659608 0.57644151 0.58972723 0.56487091
 0.56637131 0.5900429  0.58263864 0.57983322 0.58171448 0.59325561
 0.59266739 0.58533112 0.42444693 0.57935542 0.58174867 0.58332658
 0.57117211 0.5665899  0.56722756 0.58769062 0.58355268 0.5720323
 0.58895523 0.57245155 0.56809467 0.57380507 0.56666403 0.59213971
 0.58071843 0.57291861 0.5779659  0.57114638 0.58703755 0.58058691
 0.55661689 0.56934588 0.57969227 0.57932888 0.58348304 0.58388503
 0.53592341 0.56443033 0.57474484 0.50201591 0.5796591  0.57742093
 0.57560787 0.56675325 0.58483463 0.58475738 0.5702273  0.58784229
 0.5893981  0.56728936 0.60140541 0.57565564 0.57188227 0.49367947
 0.5716899  0.56805567 0.58629852 0.57267968 0.57325524 0.59336306
 0.58860722 0.56699795 0.58088526 0.57185041 0.57978539 0.58464107
 0.58043697 0.45609897 0.59473984 0.58639382 0.59348673 0.56403295
 0.58016324 0.57358967]

AUC = 0.4707, F1 score @0.5 = 0.7523




100%|██████████| 15/15 [00:03<00:00,  4.61it/s, loss=0.662]
                                                           
100%|██████████| 16/16 [01:07<00:00,  4.21s/it]A

[0.58193859 0.59566415 0.58722305 0.51594945 0.58679268 0.58813045
 0.59152398 0.57465737 0.58748816 0.57832261 0.59049008 0.56589831
 0.56793722 0.59132234 0.5837087  0.58218255 0.58268439 0.59502408
 0.59374225 0.58639042 0.42519301 0.58104451 0.58367385 0.58426897
 0.57269304 0.56793807 0.56944419 0.58923384 0.5848275  0.57299284
 0.59017975 0.57394583 0.57025597 0.57472571 0.56866872 0.5934991
 0.58223795 0.57400913 0.57950871 0.57257962 0.58811513 0.58192733
 0.55804492 0.57073108 0.5814772  0.58054066 0.58548282 0.58500364
 0.53655076 0.56557185 0.57587571 0.5027561  0.58093779 0.57881783
 0.57691784 0.56929807 0.58560707 0.58590249 0.571816   0.588551
 0.59039197 0.56837578 0.6030687  0.5767835  0.57271764 0.49433606
 0.57450921 0.56913924 0.58783949 0.57451577 0.5744375  0.59501121
 0.5901933  0.56792186 0.58219054 0.57275204 0.58058912 0.58678091
 0.58153101 0.45703487 0.59610267 0.58786048 0.59439659 0.56481402
 0.58156876 0.57436109]

AUC = 0.4736, F1 score @0.5 = 0.7523

AU

In [10]:
# Need get false prediction
# https://stackoverflow.com/questions/34698161/how-to-interpret-almost-perfect-accuracy-and-auc-roc-but-zero-f1-score-precisio